In [2]:
# Importing required libraries

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [3]:
# All the helper functions are defined here
def get_n_random_samples(measurement_df, n):
    """
    Draw n random samples without repetition from the measurement (pandas) DataFrame 
    """
    return measurement_df.sample(n, replace = False)


def SRS_sampling(step_sample_size, measurement_df, nfp, samples_df=pd.DataFrame(), decision_set = [], sample_set_size = 300):
    """
    Re-implementation of the Statistical Recursive Searching (SRS) algorithm to sample the configuration space of a software system
    (with extreme values in mind)
    This algorithm was originally proposed by Jeho OH. et al. at ESEC/FSE 2017
    """

    # First filter measurements for the current decision set (exclude any configuration that does not match the decisions in the decision set)
    filtered_df = measurement_df
    for feature, decision in decision_set:
        filtered_df = filtered_df[filtered_df[feature] == decision]

    # Check if there are still samples missing to reach the sample set size
    if not samples_df.empty and samples_df.shape[0] + step_sample_size >= sample_set_size:
        step_sample_size = sample_set_size - len(samples_df.index)
    # Draw n random samples without repetition from the filtered measurement DataFrame for the current recursive step
    new_samples_df = get_n_random_samples(filtered_df, step_sample_size) if len(filtered_df.index) > step_sample_size else filtered_df
    # Concatenate the new samples to the existing samples
    samples_df = pd.concat([samples_df, new_samples_df])

    # Sort the samples by the NFP of interest (ascending order if you are looking for best case performance, descending order if you are looking for worst case performance)
    sorted_new_samples_df = new_samples_df.sort_values(by = nfp, ascending = False)

    # Check if there are enough samples to compare at least 2 of them
    if len(sorted_new_samples_df.index) < 2:
        # If there are not enough samples to continue SRS, then sample the remaining configuration space for the missing samples
        if len(samples_df.index) < sample_set_size:
            sample_size = sample_set_size - len(samples_df.index)
            new_samples_df = get_n_random_samples(measurement_df, sample_size) if len(measurement_df.index) > sample_size else measurement_df
            samples_df = pd.concat([samples_df, new_samples_df])
        return samples_df

    # Compare the first two samples to find common decisions
    compared_first_two_samples = sorted_new_samples_df.diff().iloc[1]

    common_decisions = [] 
    for index, value in compared_first_two_samples.items():
        # Skip the NFP column (as it is not a feature decision)
        if index in nfp:
            continue
        # If the difference is 0, then the decision is common
        if value == 0:
            common_decisions.append(index)

    noteworthy_decisions = []
    # For each common decision, check if the performance is significantly different when the decision is done in one way or the other
    for candidate_decision in common_decisions:
        # Calculate the average performance for the samples where the candidate decision is enabled and disabled
        avg_performance_enabled = sorted_new_samples_df[sorted_new_samples_df[candidate_decision] == sorted_new_samples_df.iloc[0][candidate_decision]][nfp].mean()
        avg_performance_disabled = sorted_new_samples_df[sorted_new_samples_df[candidate_decision] != sorted_new_samples_df.iloc[0][candidate_decision]][nfp].mean()

        # Check of the averages are not NaN
        if avg_performance_enabled is np.nan or avg_performance_disabled is np.nan:
            continue
        # If the averages are different and not already in the decision set, then check if the difference is statistically significant
        if avg_performance_enabled != avg_performance_disabled and candidate_decision not in decision_set:
            performance_samples_enabled = sorted_new_samples_df[sorted_new_samples_df[candidate_decision] == sorted_new_samples_df.iloc[0][candidate_decision]][nfp]
            performance_samples_disabled = sorted_new_samples_df[sorted_new_samples_df[candidate_decision] != sorted_new_samples_df.iloc[0][candidate_decision]][nfp]

            welchs_ttest_result = ttest_ind(performance_samples_enabled, performance_samples_disabled, equal_var = False)
            # If the p-value is less than 0.05, then the difference is statistically significant, and the decision is added to the noteworthy decisions
            if welchs_ttest_result.pvalue < 0.05:
                noteworthy_decisions.append((candidate_decision, sorted_new_samples_df.iloc[0][candidate_decision]))
            else:
                continue
    # If there are (new) noteworthy decisions, then sample the configuration space again with the new decision set (recursively)
    if noteworthy_decisions:
        new_measurement_df = pd.concat([measurement_df, samples_df, samples_df]).drop_duplicates(keep=False)
        return SRS_sampling(step_sample_size, new_measurement_df, nfp, samples_df, decision_set = decision_set + noteworthy_decisions, sample_set_size = sample_set_size)
    else:
        # If there are no noteworthy decisions and there are still samples missing to reach the sample set size, then sample the remaining configuration space for the missing samples
        if len(samples_df.index) < sample_set_size:
            sample_size = sample_set_size - len(samples_df.index)
            new_samples_df = get_n_random_samples(measurement_df, sample_size) if len(measurement_df.index) > sample_size else measurement_df
            samples_df = pd.concat([samples_df, new_samples_df])
        return samples_df

In [4]:
# configurable system names
root_path = "./data/sampling/"

cs_information = [
    # ("7z", "Performance", ["Variable Features", "Size"], "7z (Performance)", 600),
    # ("BerkeleyDBC", "Performance", [], "Berkeley DB (Performance)", 97),
    # ("Dune", "Performance", ["Variable Features"], "Dune (Performance)", 265),
    # ("Hipacc", "Performance", ["Variable Features"], "Hipacc (Performance)", 843),
    # ("JavaGC", "Performance", ["Variable Features"], "Java GC (Performance)", 468),
    # ("LLVM", "Performance", ["MainMemory"], "LLVM (Performance)", 56),
    ("Polly", "Performance", ["Variable Features", "ElapsedTime"], "Polly (Performance)", 345),
]

# Required size of the sample set
sample_set_size = 2100
# Number of different sample sets to draw per configurable system
number_repetitions = 100

for cs, nfp_type, exclusion_list, diagram_title, t2_size in cs_information:
    print("Processing " + cs + "...")
    # read the data
    measurement_df = pd.read_csv(root_path + cs + "/measurements.csv")
    # remove the excluded columns
    measurement_df = measurement_df.drop(columns = exclusion_list)

    for i in range(1, number_repetitions + 1):
        # Set the random seed for sampling with pandas sample method (to ensure reproducibility)
        np.random.seed(i)

        # sample the configuration space using the SRS algorithm for #sample_set_size configurations
        sample_set = SRS_sampling(25, measurement_df, nfp_type, sample_set_size = sample_set_size)
        # sample the configuration space using the SRS algorithm for #t2_size configurations
        # sample_set_t2 = SRS_sampling(25, measurement_df, nfp_type, sample_set_size = t2_size)

        # remove the NFP columns for writing the sample set to a csv file
        sample_set = sample_set.drop(columns = nfp_type)
        # sample_set_t2 = sample_set_t2.drop(columns = nfp_type)

        # write the sample sets to a csv file
        sample_set.to_csv(root_path + cs + "/" + cs + "_" + str(i) + "/sampledConfigurations_SRSSampling_sampleSize_" + str(sample_set_size) + ".csv", index = False)
        # sample_set_t2.to_csv(root_path + cs + "/" + cs + "_" + str(i) + "/sampledConfigurations_SRSSampling_t2.csv", index = False)

Processing Polly...


/var/folders/hj/x93rqrkn5zx8g7nx4gdjjp0r0000gn/T/ipykernel_72021/653950964.py:68: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  welchs_ttest_result = ttest_ind(performance_samples_enabled, performance_samples_disabled, equal_var = False)
/usr/local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1253: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1253: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/var/folders/hj/x93rqrkn5zx8g7nx4gdjjp0r0000gn/T/ipykernel_72021/653950964.py:68: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unr